In [1]:
# Import standard data analysis packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#Import system/command line packages
import os
from os import path # file management
import sys
import shutil
import time # used to track code runtimes
from datetime import date # used to track date code was run

#Import scipy packages
from scipy import ndimage as ndi
from scipy import stats

# KL: Import astropy packages
from astropy.io import fits
from astropy.wcs import WCS
from astropy.io import ascii
from astropy.table import Table

# JWL: Import other packages
from math import log10, floor
from shlex import split # used to parse command line inputs
import socket # used to get computer hostname

# JWL: Import CIGALE packages
#import pcigale
#from pcigale.analysis_modules.pdf_analysis import PdfAnalysis

In [ ]:
def phangs_cigale_pipe3(self):
		"""Function to fit SEDs, with CIGALE, to associations
		
		Inputs
		------
		Distance file
		reginfo (pipe1)
		dolphot photometry (pipe2)
		
		Returns
		-------
		total_runtime: 'float'
			Runtime for step 3 of pipeline
		
		Notes
		-----
		7/6/22: JWL - config_dict can't be read by pcigale to make pcigale.ini and proceed
				JWL - works if you run 3rd step of Phangs_wshierarchy_main.py though
		7/22/22: JWL - pipe3 works now. Pipeline must be run from appropriate directory for your CIGALE installation to be recognized
		"""
		# ------------------------------
		#  Define CIGALE output folder
		# ------------------------------
		cigdir = 'out/'
		cigale_file = 'results'
		cigdir_new = self.outdir+'Cigale_outs/'
		bayes_output_path = cigdir_new+'bayesian_analysis_output/'
		# -------
		infile_SSP=datadir+'models-block-bc03-0cov.txt'
		
		# ------------------------------
		#   check for output directory
		# ------------------------------
		if (path.exists(cigdir_new)==True):
		    print('Output directory: '+cigdir_new +' already exists')
		if (path.exists(cigdir_new)==False):
		    os.mkdir(cigdir_new) 
		    print("Output Directory '% s' created" % cigdir_new)
		# ------------------------------
		
		# JWL: Initialize CIGALE stats file
		statfile_name = '{0}{1}_{2}_hpfilt{3}_Cigale_stats.txt'.format(cigdir_new,self.gal,self.tracer_band,self.filter_step)
		#statfile_name = cigdir_new+gal+'_'+tracer_band+'_hpfilt'+filter_step+'_Cigale_stats.txt'
		statfile = open(statfile_name,'w')
		statfile.write(cigdir_new+'\n')
		statfile.write('Galaxy:  {0}\n'.format(self.gal))
		statfile.write('Dolphot file:  {0}\n'.format(self.dolphot_combfile))
		statfile.write('Tracer star band: {0}\n'.format(self.tracer_band))
		statfile.write('Smoothing step: {0}\n'.format(self.filter_step))
    
		# JWL: Begin tracking runtime
		program_start_time = time.time()
		
		#---------------------------------
		# check the distance and re-set scale levels
		# Cigale step will fail is there are no regions at a given level.
		pc_scales = ['8','16','32','64']
		n_scales = len(pc_scales)
		if (self.distmpc > 17.0):
		    pc_scales = ['16','32','64']
		    n_scales = len(pc_scales)
		    min_scale_pc = 16 # minimum scale length set in parsecs
		    nlevels = n_scales #number of hierarchy levels to compute
		    
		#-------------------------------------#-------------------------------------#-------------------------------------
		#-------------------------------------#-------------------------------------#-------------------------------------
		#read in model data
		#;;--------
		#;; BC03 models
		#readcol,'models-block-bc03-0cov.txt',
		#format='(i,d,d,d,d,d,d,d,d,d,d,d,d,d,d,d)',bc03id ,bc03F275W_1 ,bc03F275W_2 ,bc03F336W_1 ,bc03F336W_2 ,bc03F435W_acs ,
		#bc03F438W_1 ,bc03F438W_2 ,bc03F555W_ACS ,bc03F555W_1 ,bc03F555W_2 ,bc03F814W_ACS ,bc03F814W_1 ,bc03F814W_1 ,bc03mstar,bc03age
		#;; convert mjy to vega mag
		#infile_SSP='models-block-bc03-0cov.txt'
		#bc03=pd.read_csv(infile_SSP, header=0,converters={'clumpSFRerr':np.float,'age_err':np.float,'mass_err':np.float}, delim_whitespace= True)
		bc03=pd.read_csv(infile_SSP, header=0, delim_whitespace= True)
		
		#filter_list = ['F336W', 'F435W', 'F555W', 'F814W']
		#ab_zp = [24.52, 25.689, 25.68, 24.99]
		#vega_zp = [23.34, 25.784, 25.71, 24.57]
		
		filter_list = ['F336W', 'F438W', 'F555W', 'F814W']
		ab_zp = [24.7074, 24.8565, 25.8242, 25.1399]
		vega_zp = [23.5265, 24.9940, 25.8325, 24.6980]
		
		bc03_U_ab = -2.5 * np.log10( bc03['F336W_UVIS_CHIP1']* np.power(10.,-29) ) - 48.60
		bc03_U_vega = bc03_U_ab + (vega_zp[0] - ab_zp[0])
		
		bc03_B_ab = -2.5 * np.log10( bc03['F438W_UVIS_CHIP1']*np.power(10.,-29) ) -48.60
		bc03_B_vega = bc03_B_ab + (vega_zp[1] - ab_zp[1])
		  
		bc03_V_ab = -2.5 * np.log10( bc03['F555W_UVIS_CHIP1']* np.power(10.,-29) ) -48.60
		bc03_V_vega = bc03_V_ab + (vega_zp[2] - ab_zp[2])
		
		bc03_I_ab = -2.5 * np.log10( bc03['F814W_UVIS_CHIP1']* np.power(10.,-29) ) -48.60
		bc03_I_vega = bc03_I_ab + (vega_zp[3] - ab_zp[3])
		#;;--------
		
		#read in combined dolphot + region info table
		#i=0
		statfile.write('pc level, med Age (.25Q,.75Q), med Mass (.25Q,.75Q), med E(B-V) (.25Q,.75Q), med nDolphot (.25Q,.75Q), med NUV dolMag (.25Q,.75Q), med M_V abs(Mag) (.25Q,.75Q) \n')
		
		for i in range(nlevels):
			
			#cig_input_file = '{0}_{1}_ws{2}pc_hpfilt{3}_Dolphot_mjy_cigfile.tab'.format(self.gal,self.tracer_band,pc_scales[i],str(self.filter_step))
			
			#cig_input_file = self.gal+'_'+self.tracer_band+'_ws'+pc_scales[i]+'pc_hpfilt'+str(self.filter_step)+'_Dolphot_mjy_cigfile.tab'
			#print(self.outdir2+cig_input_file)
			#cig_input = pd.read_csv(self.outdir2+cig_input_file,sep=' ')
			cig_input_file = self.gal+'_'+self.tracer_band+'_ws'+pc_scales[i]+'pc_hpfilt'+str(self.filter_step)+'_Dolphot_mjy_cigfile.tab'
			cig_input = pd.read_csv(self.outdir2+cig_input_file,sep=' ')
			cig_input_col = list(cig_input.columns.values)
			cig_input_filters = cig_input_col[3:13]
			print(cig_input_filters)
			print([cig_input_filters[0], cig_input_filters[2], cig_input_filters[4], cig_input_filters[6],
						                                             cig_input_filters[8]])
			datafile_name = '{0}{1}_nuv_ws{2}.fits'.format(self.outdir,self.gal,pc_scales[i])
			#datafile_name =outdir+gal+'_nuv_ws'+pc_scales[i]+'.fits'
			#gal_data_tbl = Table.read(datafile_name, format='fits')

			reginfo_file = '{0}{1}_{2}_ws{3}pc_hpfilt{4}_reginfo.fits'.format(self.outdir,self.gal,self.tracer_band,pc_scales[i],str(self.filter_step))
			#reginfo_file = outdir+gal+'_'+tracer_band+'_ws'+pc_scales[i]+'pc_hpfilt'+str(filter_step)+'_reginfo.fits'
			#reginfo_tbl = fits.getdata(reginfo_file,1)
			reginfo_tbl = Table.read(reginfo_file, format='fits')
			reginfo_df = reginfo_tbl.to_pandas()

			#df = tbl.to_pandas()
			#print(len(df.regID))
			photfile = '{0}{1}_{2}_ws{3}pc_hpfilt{4}_Dolphot.fits'.format(self.outdir2,self.gal,self.tracer_band,pc_scales[i],str(self.filter_step))
			#photfile = outdir2+gal+'_'+tracer_band+'_ws'+pc_scales[i]+'pc_hpfilt'+str(filter_step)+'_Dolphot'+'.fits'
			phot_tbl = Table.read(photfile, format='fits')
			phot_df = phot_tbl.to_pandas()
			
			#------------------------------------
			#change fitting to CHIP1
			config_dict = {'data_file': self.outdir2+cig_input_file,
		                       'parameters_file': '', 'sed_modules': ['sfh2exp', 'bc03', 'dustext', 'redshifting'],
		                       'analysis_method': 'pdf_analysis', 'cores': 8,
		                       'bands': cig_input_filters, 'properties': [],
		                       'sed_modules_params': {'sfh2exp': {'tau_main': [0.001], 'tau_burst': [0.001], 'f_burst': [0.0],
		                                                              'age': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
		                                                                          14, 15, 16, 18, 19, 21, 22, 24, 26, 28, 30,
		                                                                          32, 34, 37, 40, 43, 46, 49, 53, 57, 61, 66,
		                                                                          71, 76, 82, 88, 95, 102, 110, 118, 127, 136,
		                                                                          147, 158, 169, 182, 196, 210, 226, 243, 261,
		                                                                          281, 302, 324, 349, 375, 403, 433, 465, 500,
		                                                                          537, 577, 621, 667, 717, 770, 828, 890, 956,
		                                                                          1028, 1105, 1187, 1276, 1371, 1474, 1584,
		                                                                          1702, 1829, 1966, 2113, 2271, 2440, 2623,
		                                                                          2819, 3029, 3255, 3499, 3760, 4041, 4343,
		                                                                          4667, 5015, 5390, 5793, 6225, 6690, 7190,
		                                                                          7727, 8304, 8925, 9591, 10308, 11077,
		                                                                          11905, 12794, 13750],
		                                                              'burst_age': [1], 'sfr_0': [1.0], 'normalise': True},
		                                                  'bc03': {'imf': [1], 'metallicity': [0.02], 'separation_age': [10]},
		                                                  'dustext': {'E_BV': [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07,
		                                                                           0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15,
		                                                                           0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23,
		                                                                           0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31,
		                                                                           0.32, 0.33, 0.34, 0.35000000000000003, 0.36, 0.37,
		                                                                           0.38, 0.39, 0.4, 0.41000000000000003, 0.42, 0.43,
		                                                                           0.44, 0.45, 0.46, 0.47000000000000003, 0.48, 0.49, 0.5,
		                                                                           0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.5700000000000001,
		                                                                           0.58, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
		                                                                           0.67, 0.68, 0.6900000000000001, 0.7000000000000001,
		                                                                           0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79,
		                                                                           0.8, 0.81, 0.8200000000000001, 0.8300000000000001, 0.84,
		                                                                           0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93,
		                                                                           0.9400000000000001, 0.9500000000000001, 0.96, 0.97,
		                                                                           0.98, 0.99, 1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06,
		                                                                           1.07, 1.08, 1.09, 1.1, 1.11, 1.12, 1.1300000000000001,
		                                                                           1.1400000000000001, 1.1500000000000001, 1.16, 1.17,
		                                                                           1.18, 1.19, 1.2, 1.21, 1.22, 1.23, 1.24, 1.25, 1.26,
		                                                                           1.27, 1.28, 1.29, 1.3, 1.31, 1.32, 1.33, 1.34, 1.35, 1.36,
		                                                                           1.37, 1.3800000000000001, 1.3900000000000001,
		                                                                           1.4000000000000001, 1.41, 1.42, 1.43, 1.44, 1.45,
		                                                                           1.46, 1.47, 1.48, 1.49, 1.5], 'Rv': [3.1], 'law': [0],
		                                                                  'filters': cig_input_filters[0]+' & '+cig_input_filters[2]+' & '+cig_input_filters[4]+' & '+
		                                                                  cig_input_filters[6]+' & '+cig_input_filters[8]},
		                                                  'redshifting': {'redshift': [0.0]}},
		                       'analysis_params': {'variables': ['stellar.age', 'stellar.m_star', 'attenuation.E_BV', 'sfh.age'],
		                                               'bands': [cig_input_filters[0], cig_input_filters[2], cig_input_filters[4], cig_input_filters[6],
		                                                             cig_input_filters[8]],
		                                                             'save_best_sed': True, 'save_chi2': 'all', 'lim_flag': False,
		                                               'mock_flag': False, 'redshift_decimals': 2, 'blocks': 1}}
			#-------------------------------------#-------------------------------------#-------------------------------------
			#
			pdf = PdfAnalysis()
			pdf.process(config_dict)
			#-------------------------------------#-------------------------------------
		    
			#copy cigale output to data output dir.
			#cigale_new_file = cigdir_new+gal+'_ws'+scales[i]+'_cig_results'

			cigale_new_file = cigdir_new+gal+'_'+tracer_band+'_ws'+pc_scales[i]+'pc_hpfilt'+str(filter_step)+'_cig_results'

			#os.system('cp '+cigdir+cigale_file+'.fits '+cigale_new_file+'.fits')
			#os.system('cp '+cigdir+cigale_file+'.txt '+cigale_new_file+'.txt')
			#os.system('cp '+cigdir+'observations.txt '+cigdir_new )
			#os.system('cp '+cigdir+'observations.fits '+cigdir_new )


			#in Myr( *10^6) for ages
			#in units of Msolar

			#-------------------------------------#-------------------------------------
			#        Read in cigale data
			#cigale=fits.getdata(cigdir+cigale_file+'.fits')
			#cigale.dtype

			cigale_tbl = Table.read(cigdir+cigale_file+'.fits', format='fits')
			cigale_df = cigale_tbl.to_pandas()
		
			#-------------------------------------#-------------------------------------#-------------------------------------
			#sinan ERROR code:
			age_error, mass_error, ebv_error = [], [], []
		    
			for sed_object_index, sed_object_regID in enumerate(reginfo_df['regID'].values):
				#print(sed_object_index, sed_object_regID)

				age_file = 'out/'+str(sed_object_regID)+'_sfh.age_chi2-block-0.npy'
				mass_file = 'out/'+str(sed_object_regID)+'_stellar.m_star_chi2-block-0.npy'
				ebv_file = 'out/'+str(sed_object_regID)+'_attenuation.E_BV_chi2-block-0.npy'
		
				if (path.exists(age_file) & path.exists(mass_file) & path.exists(ebv_file)):
					models, pdf, generated_red_chi2 = self.read_data(age_file, mass_file, ebv_file, return_red_chi2=True)
					age_1sigma, mass_1sigma, ebv_1sigma = self.calc_chi2_uncertainty(models[:, 0], models[:, 1], models[:, 2], generated_red_chi2)

					age_error.append(age_1sigma)
					mass_error.append(mass_1sigma)
					ebv_error.append(ebv_1sigma)
				else:
					age_error.append(np.NAN)
					mass_error.append(np.NAN)
					ebv_error.append(np.NAN)
			
			"""KEEP MASS-AGE-EBV WITH 3 SIGNIFICANT FIGURES"""
			cigale_df['reg_dolflux_Age_MinChiSq_err'] = np.round(age_error, 3)
			cigale_df['reg_dolflux_Mass_MinChiSq_err'] = np.round(mass_error, 3)
			cigale_df['reg_dolflux_Ebv_MinChiSq_err'] = np.round(ebv_error, 3)

			#-----------
			#save cigale output to data table
			#-----------
			cigale_df.to_csv(cigale_new_file+'.csv')
			new_cigale_tbl = Table.from_pandas(cigale_df)
			new_cigale_tbl.write(cigale_new_file+'.fits',overwrite=True)

			#-----------
			shutil.move(cigdir+'observations.txt',cigdir_new+'observations.txt')
			shutil.move(cigdir+'observations.fits',cigdir_new+'observations.fits')

			#os.system('cp '+cigdir+cigale_file+'.fits '+cigale_new_file+'.fits')
			#os.system('cp '+cigdir+cigale_file+'.txt '+cigale_new_file+'.txt')
			#os.system('cp '+cigdir+'observations.txt '+cigdir_new )
			#os.system('cp '+cigdir+'observations.fits '+cigdir_new )

			#-------------------------------------#-------------------------------------#-------------------------------------
			#"""Call to Jordan Turner's Bayesian Priors analysis here"""

			#   ---------------      ADD Bayesian CODE HERE      ---------------   #

			#-------------------------------------#-------------------------------------#-------------------------------------
			#If using mac or linux can remove cigale directory when done.
			# Will NOT run on a windows OS
			#remove cigale directory
			os.system('rm -r '+cigdir)

			#------------------------------------    #------------------------------------    #------------------------------------
			g_mass = (np.log10(cigale_tbl['best.stellar.m_star']) > 0 )
			Mv = phot_df.v_dolmag - self.dist_mod

			#Check for Inf
			Mv[(Mv==np.inf)] = np.NAN
			phot_df.nuv_dolmag[(phot_df.nuv_dolmag==np.inf)] = np.NAN
		    
			print(pc_scales[i]+'pc, '+str((np.nanmedian(np.log10(cigale_df['best.stellar.age_m_star'][g_mass]*1e6))) )
				+' ('+str((np.nanquantile(np.log10(cigale_df['best.stellar.age_m_star'][g_mass]*1e6), .25)))+','
				+str((np.nanquantile(np.log10(cigale_df['best.stellar.age_m_star'][g_mass]*1e6), .75)))+') &'
				+str((np.nanmedian(np.log10(cigale_df['best.stellar.m_star'][g_mass]))))
				+' ('+str((np.nanquantile(np.log10(cigale_df['best.stellar.m_star'][g_mass]), .25)))+','+str((np.nanquantile(np.log10(cigale_df['best.stellar.m_star'][g_mass]), .75)))+') &'
				+str((np.nanmedian(cigale_df['best.attenuation.E_BV'][g_mass])))
				+' ('+str((np.nanquantile(cigale_df['best.attenuation.E_BV'][g_mass], .25)))+','+str((np.nanquantile(cigale_df['best.attenuation.E_BV'][g_mass], .75)))+') &'
				+str((np.nanmedian(phot_df.ndolphot)))+' ('+str((np.nanquantile(phot_df.ndolphot, .25)))+','+str((np.nanquantile(phot_df.ndolphot, .75)))+') &'
				+str((np.nanmedian(phot_df.nuv_dolmag)))+' ('+str((np.nanquantile(phot_df.nuv_dolmag, .25)))+','+str((np.nanquantile(phot_df.nuv_dolmag, .75)))+') &'
				+str((np.nanmedian(Mv)))+' ('+str((np.nanquantile(Mv, .25)))+','+str((np.nanquantile(Mv, .75)))+') '
				)

			#------------------------------------    #------------------------------------    #------------------------------------
			#       Save stats
			#------------------------------------
			#g_mass = (np.log10(cigale_tbl['best.stellar.m_star']) > 0 )
			#Mv = phot_df.v_dolmag - dist_mod
			# med reddening, ndolphot,med '+tracer_band+'(.25Q,.75Q)   \n')
			statfile.write(pc_scales[i]+'pc, '+str(self.round_sig(np.nanmedian(np.log10(cigale_df['best.stellar.age_m_star'][g_mass]*1e6))) )
						+' ('+str(self.round_sig(np.nanquantile(np.log10(cigale_df['best.stellar.age_m_star'][g_mass]*1e6), .25)))+','
						+str(self.round_sig(np.nanquantile(np.log10(cigale_df['best.stellar.age_m_star'][g_mass]*1e6), .75)))+') &'
						+str(self.round_sig(np.nanmedian(np.log10(cigale_df['best.stellar.m_star'][g_mass]))))
						+' ('+str(self.round_sig(np.nanquantile(np.log10(cigale_df['best.stellar.m_star'][g_mass]), .25)))+','
						+str(self.round_sig(np.nanquantile(np.log10(cigale_df['best.stellar.m_star'][g_mass]), .75)))+') &'
						+str((np.nanmedian(cigale_df['best.attenuation.E_BV'][g_mass])))
						+' ('+str(np.nanquantile(cigale_df['best.attenuation.E_BV'][g_mass], .25))
						+','+str(np.nanquantile(cigale_df['best.attenuation.E_BV'][g_mass], .75))+') &'
						+str(self.round_sig(np.nanmedian(phot_df.ndolphot)))+' ('+str(self.round_sig(np.nanquantile(phot_df.ndolphot, .25)))
						+','+str(self.round_sig(np.nanquantile(phot_df.ndolphot, .75)))+') &'
						+str(self.round_sig(np.nanmedian(phot_df.nuv_dolmag)))+' ('+str(self.round_sig(np.nanquantile(phot_df.nuv_dolmag, .25)))
						+','+str(self.round_sig(np.nanquantile(phot_df.nuv_dolmag, .75)))+') &'
						+str(self.round_sig(np.nanmedian(Mv)))+' ('+str(self.round_sig(np.nanquantile(Mv, .25)))+','+str(self.round_sig(np.nanquantile(Mv, .75)))+') '
						+' \n' )
						
			#------------------------------------    #------------------------------------    #------------------------------------
			#Save combined datafile
			#--------------------------------
			t_comb =Table([reginfo_df['regID'],reginfo_df['peak_xpos'],reginfo_df['peak_ypos'],reginfo_df['peak_ra'],reginfo_df['peak_dec'],reginfo_df['area_pc'],reginfo_df['rad_pc'],
							phot_df.nuv_dolflux_mjy, phot_df.nuv_dolflux_mjy_err ,phot_df.u_dolflux_mjy,phot_df.u_dolflux_mjy_err, phot_df.b_dolflux_mjy, phot_df.b_dolflux_mjy_err,
							phot_df.v_dolflux_mjy, phot_df.v_dolflux_mjy_err, phot_df.i_dolflux_mjy, phot_df.i_dolflux_mjy_err,
							phot_df.nuv_dolmag, phot_df.nuv_dolmagUnc, phot_df.u_dolmag, phot_df.u_dolmagUnc, phot_df.b_dolmag, phot_df.b_dolmagUnc,
							phot_df.v_dolmag, phot_df.v_dolmagUnc, phot_df.i_dolmag, phot_df.i_dolmagUnc,
							cigale_df['best.stellar.age_m_star'], cigale_df['reg_dolflux_Age_MinChiSq_err'],
							cigale_df['best.stellar.m_star'], cigale_df['reg_dolflux_Mass_MinChiSq_err'],
							cigale_df['best.attenuation.E_BV'], cigale_df['reg_dolflux_Ebv_MinChiSq_err'], cigale_df['best.reduced_chi_square']
		                        ],
							names=('reg_id','reg_x','reg_y','reg_ra','reg_dec','reg_area','reg_rad',
		                             'NUV_dolflux_mjy','NUV_dolflux_mjy_err','U_dolflux_mjy','U_dolflux_mjy_err','B_dolflux_mjy','B_dolflux_mjy_err',
		                             'V_dolflux_mjy','V_dolflux_mjy_err','I_dolflux_mjy', 'I_dolflux_mjy_err',            										  
		                             'NUV_dolmag_vega','NUV_dolmag_vega_err','U_dolmag_vega','U_dolmag_vega_err',
		                             'B_dolmag_vega','B_dolmag_vega_err','V_dolmag_vega','V_dolmag_vega_err',
									 'I_dolmag_vega','I_dolmag_vega_err',
		                             'reg_dolflux_Age_MinChiSq','reg_dolflux_Age_MinChiSq_err',
		                             'reg_dolflux_Mass_MinChiSq','reg_dolflux_Mass_MinChiSq_err',
		                             'reg_dolflux_Ebv_MinChiSq', 'reg_dolflux_Ebv_MinChiSq_err','reg_dolflux_ChiSq_Reduced'),
							meta={'ws_data':gal+' tracer band:'+tracer_band+'scale level [pc]:'+pc_scales[i] })
			
			combfile = '{0}_phangshst_associations_{1}_ws{2}pc_v1'.format(self.gal,self.tracer_band,pc_scales[i])
			#combfile = gal+'_phangshst_associations_'+tracer_band+'_ws'+pc_scales[i]+'pc_v1'
			#os.system('rm '+outdir2+photfile+'.fits')
			fits.writeto(self.outdir2+combfile+'.fits', np.array(t_comb),overwrite=True)
			df_comb = t_comb.to_pandas()
			df_comb.to_csv(self.outdir2+combfile+'.csv')
			print(self.outdir2+combfile+'.csv written' )
			#------------------------------------ 

			#--------------------------------
		statfile.close()
		#--------------------------------
		print('%s --- Cigale(pipe3) finished...' % gal)
		
		total_runtime = (time.time() - program_start_time)/60.0
		print("%s --- Total time --- %.2f seconds (%.2f minutes) ---" % (gal,time.time() - program_start_time,total_runtime))
		
		return(np.round(total_runtime,2))